# Iterative EndPoint Fit Algorithm

## Dependencies

1. NumPy
2. SciPy
3. MatplotLib
4. Pandas
5. Math

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

## Representasi Titik

$P=\{x,y\}$

In [ ]:
p0 = np.array([10,5])

In [ ]:
p0

## Representasi Garis

$\overleftrightarrow{AB}$

$L = \{P_0,..,P_n\}$


In [ ]:
p0 = np.array([0,0])
p1 = np.array([10,10])
l0 = np.array([p0,p1])

In [ ]:
l0

## Dataset Titik

In [ ]:
p0 = np.array([0,0])
p1 = np.array([2,2])
p2 = np.array([3,4])
p3 = np.array([4,10])

dP = np.array([p0,p1,p2,p3])

In [ ]:
dP

In [ ]:
# Menampilkan sumbu x dari dataset titik
dP[:4,:1] 

In [ ]:
# Menampilkan sumbu y dari dataset titik
dP[:4,1:] 

In [ ]:
plt.plot(dP[:4,:1], dP[:4,1:], 'ro')
plt.axis([0, 5, 0, 15])
plt.show()

Mengambil nilai koordinat titik dari file csv

In [ ]:
df = pd.read_csv('C:\Research\IEPF-Line-Extraction\Source-Code\dataset.csv')
dP = df.as_matrix()

In [ ]:
dP

In [ ]:
dP = dP[:,1:]

In [ ]:
dP

In [ ]:
# Mencari total jumlah dataset point
totP,_ = dP.shape

In [ ]:
totP

In [ ]:
plt.plot(dP[:totP,:1], dP[:totP,1:], 'ro')
plt.axis([0, 640, 0, 640])
plt.show()

In [ ]:
# Fungsi load dataset
# Input berupa file csv
# Output berupa dataset point dalam bentuk kumpulan koordinat x,y
def csvToDatasetPoint(csvFile):
    df = pd.read_csv(csvFile)
    dP = df.as_matrix()
    dP = dP[:,1:]
    return dP

In [ ]:
csvToDatasetPoint('C:\Research\IEPF-Line-Extraction\Source-Code\dataset.csv')

## Jarak antar titik
Fungsi untuk menghitung jarak antara dua titik

* Input : Koordinat titik A dan koordinat titik B

* Ouput : Jarak antara dua titik

In [ ]:
def distancePointToPoint(p0, p1):
    return math.sqrt(((p1[0]-p0[0])*(p1[0]-p0[0])) + ((p1[1]-p0[1])*(p1[1]-p0[1])))

In [ ]:
pointA = np.array([0,0])

In [ ]:
pointB = np.array([10,20])

In [ ]:
distancePointToPoint(pointA, pointB)

## Fungsi Garis

$y = mx + b$

Fungsi untuk mencari persamaan garis

* Input : Koordinat Titik 1 dan koordinat titik 2
* Output : Nilai m (gradient) dan b (constanta)

In [ ]:
# Hanya berlaku jika garis memiliki kemiringan
def lineFunction(l0):
#     Tipe garis 0 = miring, 1 = gradien = 0, -1 gradien tak hingga
    ltype = 0
#     Jika delta x = 0
    if l0[1,0] - l0[0,0] == 0:
        ltype = -1
    elif l0[1,1] - l0[0,1] == 0:
        ltype = 1
    else:
        ltype = 0

    if ltype == 0:
        m = float(l0[1,1] - l0[0,1]) / float(l0[1,0] - l0[0,0])
        b = float(l0[0,1] - m * l0[0,0])
    else:
        m = 0
        b = 0
        
    return ltype, m, b

In [ ]:
p0 = np.array([10,0])
p1 = np.array([0,0])

l0 = np.array([p0,p1])

lineFunction(l0)

Convert y=mx+b to ax+bx+c=0
https://www.youtube.com/watch?v=h13wI_gi4GA

In [ ]:
def convertToABC(ltype, m, b):
    if ltype == -1:
        _a = 0.00
        _b = 0.00
        _c = 0.00
    elif ltype == 1:
        _a = 0.00
        _b = 0.00
        _c = 0.00
    else:
        _a = m * -1.00
        _b = 1.00
        _c = b * -1.00
    return _a, _b, _c

In [ ]:
a, b, c = convertToABC(0, 2, 1.6)

In [ ]:
a

In [ ]:
b

In [ ]:
c

## Jarak titik ke garis
https://www.youtube.com/watch?v=h13wI_gi4GA

$d = |ax + by + c| / $

In [ ]:
def distancePointToLine(p0, l0):
#     Jika delta x = 0
#     if l?[1,0] - l0[0,0] != 0:
    ltype, m, b = lineFunction(l0)
#     print ltype
    _a, _b, _c = convertToABC(ltype, m, b)
    d = 0
    if ltype == -1:
        d = abs(p0[0] - l0[0,0])
    elif ltype == 1:
        d = abs(p0[1] - l0[0,1])
    else:
        d = abs(_a * p0[0] + _b * p0[1] + _c) / math.sqrt(_a * _a + _b * _b)

    return d
    

In [ ]:
# Example

p0 = np.array([2,1])
l0 = np.array([[0,100],[10,10]])

distancePointToLine(p0, l0)

## Iterative End Point Fit

### Split

Input berupa dataset point
Ouput berupa cluster point

In [ ]:
def split(threshold, dP, eP):
#     print 'Masuk IEPF'
    maxD = 0
    breakPointIndex = -1

    totalPoint, _ = dP.shape
    totalEndPoint = eP.size
    
    for i in range(0,totalEndPoint-1):
        eP0 = eP[i]
        ePN = eP[i+1] - 1 # Ini coba di cek lagi

        l0 = np.array([dP[eP0],dP[ePN]])

        for j in range(eP[i],eP[i+1]):
            tempD =  distancePointToLine(dP[j], l0)
            if tempD > threshold:
                if (tempD > maxD):
                    maxD = tempD
                    breakPointIndex = j
        
    if breakPointIndex != -1:
        eP = np.insert(eP, [totalEndPoint-1], breakPointIndex)
        eP = split(threshold, dP, eP)
        
    return eP    

In [ ]:
dP = csvToDatasetPoint('C:\Research\IEPF-Line-Extraction\Source-Code\dataset.csv')
totalP, _ = dP.shape

# print 'Jumlah Point'
# print jumlahPoint

p0 = np.array([dP[0,0],dP[0,1]])
p1 = np.array([dP[totalP-1,0],dP[totalP-1,1]])
eP = np.array([0,totalP-1])
# print p

# a = e
# print p.size
eP = split(30, dP, eP)
totalEP = eP.size
# print eP.size
# X
# print dP[eP[0:4],:1] 
# print dP[eP[0:4],1:] 
plt.plot(dP[:totalP,:1], dP[:totalP,1:], 'ro')
plt.plot(dP[eP[0:totalEP],:1], dP[eP[0:totalEP],1:])
plt.axis([0, 640, 0, 640])
plt.show()


### Merge

Input berupa minimal panjang, dataset point dan endpoint awal
Outpu berupa endpoint yang sudah di merge

In [ ]:
def merge(threshold, dP, eP):
#     print dP
    totalEndPoint = eP.size
#     print totalEndPoint-1
    # Jangan pakai in range
    i = 0
    while i < totalEndPoint-1 :
#     for i in range(0, totalEndPoint-1):
        print 'I = {}'.format(i)
        print 'eP Index = {}'.format(i)
        print 'totalEndPoint = {}'.format(totalEndPoint)
        eP0 = eP[i]
        ePN = eP[i+1] 
        
        print 'eP0 = {}'.format(eP0)
        print 'ePN = {}'.format(ePN)
        tempD = distancePointToPoint(dP[eP0], dP[ePN])
        print tempD
        # Jika jarak antara kedua titik kurang dari threshold
        if tempD < threshold:
            # Delete endpoint
            eP = np.delete(eP,[i+1])
#             totalEndPoint = totalEndPoint - 2
            # Update latest totalEndPoint
            totalEndPoint = eP.size
            print 'Merge'
        i = i + 1
        print '---------------------'
    return eP

In [ ]:
print eP

In [ ]:
eP = merge(200, dP, eP)

In [ ]:
eP

In [ ]:
plt.plot(dP[:totalP,:1], dP[:totalP,1:], 'ro')
plt.plot(dP[eP[0:totalEP],:1], dP[eP[0:totalEP],1:])
plt.axis([0, 640, 0, 640])
plt.show()

## Least Square Method

In [ ]:
import numpy as np

In [ ]:
a = np.array([[2,3],[3,4]])

In [ ]:
a

In [ ]:
a = np.array([np.array([[2,3],[3,4]]),np.array([[4,5],[3,4],[5,6]])])

In [ ]:
a

In [ ]:
a[0]

In [ ]:
a[1]

In [ ]:
a[0,1]

In [ ]:
%%cmd

dir